In [49]:
import dotenv
import pandas as pd

from typing import List

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

In [50]:
dotenv.load_dotenv()

True

# Utils

In [51]:
class EvaluationCategory(BaseModel):
    value: bool = Field(...)
    details: List[str] = Field(...)


class CodeEvaluation(BaseModel):
    no_syntax_errors: EvaluationCategory = Field(...)
    variable_usage: EvaluationCategory = Field(...)
    clarity_conciseness: EvaluationCategory = Field(...)
    user_requirements: EvaluationCategory = Field(...)

parser = JsonOutputParser(pydantic_object=CodeEvaluation)

In [52]:
def evaluation_results(evaluation: CodeEvaluation):
    data = [[
        evaluation["no_syntax_errors"]["value"],
        "\n".join(evaluation["no_syntax_errors"]["details"]),
        evaluation["variable_usage"]["value"],
        "\n".join(evaluation["variable_usage"]["details"]),
        evaluation["clarity_conciseness"]["value"],
        "\n".join(evaluation["clarity_conciseness"]["details"]),
        evaluation["user_requirements"]["value"],
        "\n".join(evaluation["user_requirements"]["details"]),
    ]]
    return data


# Rules Evaluation

In [53]:
def evaluate_rule(input: str, answer: str, context_prompt: str, type: str) -> list:
    if type != "rule":
        print(input, answer, context_prompt, type)
        raise ValueError("Type must be 'rule'")

    prompt = """
    You are an AI assistant tasked with evaluating C# code written for the Ferryt Low-Code Platform. The platform provides predefined process fields, screens, methods, and objects. Your job is to analyze the code's quality and provide detailed feedback on the following aspects:

1. **No Syntax Errors** (`no_syntax_errors`):  
   - Does the code contain any syntax errors, typos, or issues that would prevent compilation?  
   - Identify **all** mistakes, no matter how small, and explain their impact.  
   - Code with syntax errors should be considered **unacceptable**.  

2. **Variable Usage** (`variable_usage`):  
   - Are variables used **efficiently and correctly**?  
   - Point out **every** instance of redundant variables, incorrect assignments, or wasted memory.  
   - Flag **any** poor naming conventions or misleading variable usage.  
   - **Critically evaluate the handling of null values. Any failure to check for null references before usage should be flagged as a serious issue.**  
   - **If there is any possibility of a NullReferenceException occurring, it must be explicitly called out as an unacceptable risk.**  
   - Code must ensure **safe** access to nullable objects, including checking `.HasValue` before `.Value`, using null-coalescing (`??`), or applying appropriate null-check guards.  

3. **Clarity & Conciseness** (`clarity_conciseness`):  
   - Is the validation code clear, concise, and easy to understand?  
   - Provide suggestions to improve clarity, such as simplifying expressions, improving naming conventions, or adding comments if necessary.

4. **User Requirements** (`user_requirements`):  
   - Does the validation logic work as expected?  
   - Are there any logical flaws or bugs in the code that might cause incorrect validation?  

    **Return only valid JSON in the exact format below. Do not add explanations.**  
    Format:
```json
    {{
        "no_syntax_errors": {{
            "value": true,
            "details": ["message goes here", "message goes here"]
        }},
        "variable_usage": {{
            "value": true,
            "details": ["message goes here", "message goes here"]
        }},
        "clarity_conciseness": {{
            "value": true,
            "details": ["message goes here", "message goes here"]
        }},
        "user_requirements": {{
            "value": true,
            "details": ["message goes here", "message goes here"]
        }}
    }}
```
    ---

    ### Context:
    {context}

    ---


    ### User Input:
    {user_input}

    ---

    ### Generated Code:
    {generated_code}
    """

    prompt = prompt.replace("{", "{{").replace("}", "}}")
    prompt = prompt.format(generated_code=answer, user_input=input, context=context_prompt)
    llm = ChatOpenAI(temperature=0.0, model="gpt-4")
    llm_chain = PromptTemplate.from_template(prompt) | llm | parser
    response = llm_chain.invoke(
        input={
            "question": "Evaluate the code.",
            "user_input": input,
            "context": context_prompt,
            "generated_code": answer,
        }
    )
    return evaluation_results(response)

In [54]:
test_cases = [
    {
        "input": "Ustaw odrzucenie przez wiek na true, jeżeli jeden członek rodziny jest niepełnoletni.",
        "response": """
if (PF.UR_DaneRodziny_T.Any(x => x.Wiek.HasValue && x.Wiek.Value < 18))
{
    PF.UR_OdrzuceniePrzezWiek = true;
}
        """,
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

# Universal Prompt for C# Code Generation for Client Data and Screens Management

When writing C# code, use the available process fields and screens to manage client data and user interface. Follow these guidelines:

## 1. Available Process Fields:

- PF.UR_OdrzuceniePrzezWiek: Bool
- PF.UR_DaneKlienta_S.Imie: String
- PF.UR_DaneKlienta_S.Nazwisko: String
- PF.UR_DaneKlienta_S.Email: String
- PF.UR_DaneKlienta_S.PESEL: String
- PF.UR_DaneRodziny_T[].Imie: String
- PF.UR_DaneRodziny_T[].Nazwisko: String
- PF.UR_DaneRodziny_T[].Wiek: Int



### Properties:
Fields have the following properties:
- `HasValue`, `IsEditable`, `IsRequired`, `IsVisible`, `Value`.

### Actions:
You can perform these operations on the fields:
- `SetEditable(bool)`, `SetVisible(bool)`, `SetRequired(bool)`, `SetNull()`, `GetValueOrDefault(defaultValue)`.

### Arrays:
- Arrays are complex datatypes that store fields or structures. Their name should end with `_T`.
    A specific element in the array can be accessed via `[index]` or `.Items(index)`.
- Operations on arrays: Length, MinimumSize (sets minimum size of an array eg. PF.Array.MinimumSize = 1)

## 2. Available Screens:
- `E010_Powitanie`
- `E020_DaneRodziny`
- `E030_WyborUbezpieczenia`
- `E040_Zgody`
- `E050_Podsumowanie`
- `E060_Odrzucenie`
- `E_techMessage`
- `Tech_SessionEndScreen`
- `Tech_TopScreen`
- `Tech_BottomScreen`
- `Tech_TabsScreen`


Do not use screen variables unless specifically asked to by user.

## Methods:
Screens can be accessed by `G.`.
Screens can be controlled using the following methods:
- `Hide()`, `HideAll()`, `Show()`, `ShowAll()`.

## 3. Accessing User Data:
- `USER.Current` provides access to properties like `IsAuthenticated`, `UserLogin`, `UserFullName`, `UserEmail`,
    `UserID`, etc.
    
## 4. ENV Object:
- ENV gives access to many environment variables like:
    * ActionIndices[0] - points to the element on which the action was last performed (in arrays)
    * ApplicationGuid
    * UserDevice.IsMobile
    * UserDevice.BrowserInfo
    * UserDevice.OperatingSystem etc.
    
## 5. ACTION object
- ACTION object has methods like: `ClearDocumentRequirement()`, `ClearFieldRequirement()`,
    `SetFieldsReadOnly()` - disables editable fields, `VisibilityFlagHide(flagName)` - hide all actions with the
    flag set, `VisibilityFlagHideAll()`, `VisibilityFlagShow(flagName)` - show all actions with the flag set`,
    `VisibilityFlagShowAll()`
   
## 6. Example Functionalities:
- Setting field values based on conditions.
- Controlling field visibility, editability and requirements.
- Assigning user data to client data under specific conditions.
- Displaying or hiding screens.

## 7. Notes:
- To assign a value to a specific field, use the '=' symbol, e.g., PF.FieldInt1 = 10.
- Reference to the `.Value` property, may cause an error if the field is null.
    To avoid this, first check that the value is via `.HasValue`.
- Write a simple and easy-to-understand code. Avoid using advanced or overly complex syntax. Stick to basic structures
    and clear logic so that the code is accessible even to individuals with limited programming skills.
- Do not use comments in the code.

Always remember: respond with code only, unless instructed otherwise.

## Query:
Set the age rejection to true if one family member is a minor.
        """,
        "type": "rule",
    },
    {
        "input": "Sprawdź czy w rodzinie istnieje osoba starsza niż 80 lat, jeśli tak zmień składkę podstawową na 100 * ilość osób w rodzinie.",
        "response": """
if (PF.UR_DaneRodziny_T.Any(x => x.Wiek.HasValue && x.Wiek.Value > 80))
{
    PF.UR_Skladka = 100 * PF.UR_DaneRodziny_T.Length;
}
""",
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

# Universal Prompt for C# Code Generation for Client Data and Screens Management

When writing C# code, use the available process fields and screens to manage client data and user interface. Follow these guidelines:

## 1. Available Process Fields:

- PF.UR_OdrzuceniePrzezWiek: Bool
- PF.UR_Skladka: Decimal
- PF.UR_DaneRodziny_T[].Imie: String
- PF.UR_DaneRodziny_T[].Nazwisko: String
- PF.UR_DaneRodziny_T[].Wiek: Int
- PF.UR_DaneKlienta_S.Imie: String
- PF.UR_DaneKlienta_S.Nazwisko: String
- PF.UR_DaneKlienta_S.Email: String
- PF.UR_DaneKlienta_S.PESEL: String



### Properties:
Fields have the following properties:
- `HasValue`, `IsEditable`, `IsRequired`, `IsVisible`, `Value`.

### Actions:
You can perform these operations on the fields:
- `SetEditable(bool)`, `SetVisible(bool)`, `SetRequired(bool)`, `SetNull()`, `GetValueOrDefault(defaultValue)`.

### Arrays:
- Arrays are complex datatypes that store fields or structures. Their name should end with `_T`.
    A specific element in the array can be accessed via `[index]` or `.Items(index)`.
- Operations on arrays: Length, MinimumSize (sets minimum size of an array eg. PF.Array.MinimumSize = 1)

## 2. Available Screens:
- `E010_Powitanie`
- `E020_DaneRodziny`
- `E030_WyborUbezpieczenia`
- `E040_Zgody`
- `E050_Podsumowanie`
- `E060_Odrzucenie`
- `E_techMessage`
- `Tech_SessionEndScreen`
- `Tech_TopScreen`
- `Tech_BottomScreen`
- `Tech_TabsScreen`


Do not use screen variables unless specifically asked to by user.

## Methods:
Screens can be accessed by `G.`.
Screens can be controlled using the following methods:
- `Hide()`, `HideAll()`, `Show()`, `ShowAll()`.

## 3. Accessing User Data:
- `USER.Current` provides access to properties like `IsAuthenticated`, `UserLogin`, `UserFullName`, `UserEmail`,
    `UserID`, etc.
    
## 4. ENV Object:
- ENV gives access to many environment variables like:
    * ActionIndices[0] - points to the element on which the action was last performed (in arrays)
    * ApplicationGuid
    * UserDevice.IsMobile
    * UserDevice.BrowserInfo
    * UserDevice.OperatingSystem etc.
    
## 5. ACTION object
- ACTION object has methods like: `ClearDocumentRequirement()`, `ClearFieldRequirement()`,
    `SetFieldsReadOnly()` - disables editable fields, `VisibilityFlagHide(flagName)` - hide all actions with the
    flag set, `VisibilityFlagHideAll()`, `VisibilityFlagShow(flagName)` - show all actions with the flag set`,
    `VisibilityFlagShowAll()`
   
## 6. Example Functionalities:
- Setting field values based on conditions.
- Controlling field visibility, editability and requirements.
- Assigning user data to client data under specific conditions.
- Displaying or hiding screens.

## 7. Notes:
- To assign a value to a specific field, use the '=' symbol, e.g., PF.FieldInt1 = 10.
- Reference to the `.Value` property, may cause an error if the field is null.
    To avoid this, first check that the value is via `.HasValue`.
- Write a simple and easy-to-understand code. Avoid using advanced or overly complex syntax. Stick to basic structures
    and clear logic so that the code is accessible even to individuals with limited programming skills.
- Do not use comments in the code.

Always remember: respond with code only, unless instructed otherwise.

## Query:
Check if there is a person older than 80 years in the family; if so, change the basic contribution to 100 times the number of people in the family.
""",
        "type": "rule",
    },
    {
        "input": "Ustaw minimalną długość tablicy z danymi rodziny na 1.",
        "response": """
PF.UR_DaneRodziny_T.MinimumSize = 1;
""",
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

# Universal Prompt for C# Code Generation for Client Data and Screens Management

When writing C# code, use the available process fields and screens to manage client data and user interface. Follow these guidelines:

## 1. Available Process Fields:

- PF.UR_DaneRodziny_T[].Imie: String
- PF.UR_DaneRodziny_T[].Nazwisko: String
- PF.UR_DaneRodziny_T[].Wiek: Int



### Properties:
Fields have the following properties:
- `HasValue`, `IsEditable`, `IsRequired`, `IsVisible`, `Value`.

### Actions:
You can perform these operations on the fields:
- `SetEditable(bool)`, `SetVisible(bool)`, `SetRequired(bool)`, `SetNull()`, `GetValueOrDefault(defaultValue)`.

### Arrays:
- Arrays are complex datatypes that store fields or structures. Their name should end with `_T`.
    A specific element in the array can be accessed via `[index]` or `.Items(index)`.
- Operations on arrays: Length, MinimumSize (sets minimum size of an array eg. PF.Array.MinimumSize = 1)

## 2. Available Screens:
- `E010_Powitanie`
- `E020_DaneRodziny`
- `E030_WyborUbezpieczenia`
- `E040_Zgody`
- `E050_Podsumowanie`
- `E060_Odrzucenie`
- `E_techMessage`
- `Tech_SessionEndScreen`
- `Tech_TopScreen`
- `Tech_BottomScreen`
- `Tech_TabsScreen`


Do not use screen variables unless specifically asked to by user.

## Methods:
Screens can be accessed by `G.`.
Screens can be controlled using the following methods:
- `Hide()`, `HideAll()`, `Show()`, `ShowAll()`.

## 3. Accessing User Data:
- `USER.Current` provides access to properties like `IsAuthenticated`, `UserLogin`, `UserFullName`, `UserEmail`,
    `UserID`, etc.
    
## 4. ENV Object:
- ENV gives access to many environment variables like:
    * ActionIndices[0] - points to the element on which the action was last performed (in arrays)
    * ApplicationGuid
    * UserDevice.IsMobile
    * UserDevice.BrowserInfo
    * UserDevice.OperatingSystem etc.
    
## 5. ACTION object
- ACTION object has methods like: `ClearDocumentRequirement()`, `ClearFieldRequirement()`,
    `SetFieldsReadOnly()` - disables editable fields, `VisibilityFlagHide(flagName)` - hide all actions with the
    flag set, `VisibilityFlagHideAll()`, `VisibilityFlagShow(flagName)` - show all actions with the flag set`,
    `VisibilityFlagShowAll()`
   
## 6. Example Functionalities:
- Setting field values based on conditions.
- Controlling field visibility, editability and requirements.
- Assigning user data to client data under specific conditions.
- Displaying or hiding screens.

## 7. Notes:
- To assign a value to a specific field, use the '=' symbol, e.g., PF.FieldInt1 = 10.
- Reference to the `.Value` property, may cause an error if the field is null.
    To avoid this, first check that the value is via `.HasValue`.
- Write a simple and easy-to-understand code. Avoid using advanced or overly complex syntax. Stick to basic structures
    and clear logic so that the code is accessible even to individuals with limited programming skills.
- Do not use comments in the code.

Always remember: respond with code only, unless instructed otherwise.

## Query:
 Set the minimum length of the family data array to 1.
""",
        "type": "rule",
    },
    {
        "input": "Jeśli kod zniżkowy ma wartość \"UAM5\" to obniż składkę o 5 procent i wyświetl pole ze składką.",
        "response" : """
if (PF.UR_KodZnizkowy.HasValue && PF.UR_KodZnizkowy.Value == "UAM5")
{
    PF.UR_Skladka = PF.UR_Skladka * 0.95M;
    PF.UR_Skladka.SetVisible(true);
}
""",
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

# Universal Prompt for C# Code Generation for Client Data and Screens Management

When writing C# code, use the available process fields and screens to manage client data and user interface. Follow these guidelines:

## 1. Available Process Fields:

- PF.UR_KodZnizkowy: String
- PF.UR_Skladka: Decimal



### Properties:
Fields have the following properties:
- `HasValue`, `IsEditable`, `IsRequired`, `IsVisible`, `Value`.

### Actions:
You can perform these operations on the fields:
- `SetEditable(bool)`, `SetVisible(bool)`, `SetRequired(bool)`, `SetNull()`, `GetValueOrDefault(defaultValue)`.

### Arrays:
- Arrays are complex datatypes that store fields or structures. Their name should end with `_T`.
    A specific element in the array can be accessed via `[index]` or `.Items(index)`.
- Operations on arrays: Length, MinimumSize (sets minimum size of an array eg. PF.Array.MinimumSize = 1)

## 2. Available Screens:
- `E010_Powitanie`
- `E020_DaneRodziny`
- `E030_WyborUbezpieczenia`
- `E040_Zgody`
- `E050_Podsumowanie`
- `E060_Odrzucenie`
- `E_techMessage`
- `Tech_SessionEndScreen`
- `Tech_TopScreen`
- `Tech_BottomScreen`
- `Tech_TabsScreen`


Do not use screen variables unless specifically asked to by user.

## Methods:
Screens can be accessed by `G.`.
Screens can be controlled using the following methods:
- `Hide()`, `HideAll()`, `Show()`, `ShowAll()`.

## 3. Accessing User Data:
- `USER.Current` provides access to properties like `IsAuthenticated`, `UserLogin`, `UserFullName`, `UserEmail`,
    `UserID`, etc.
    
## 4. ENV Object:
- ENV gives access to many environment variables like:
    * ActionIndices[0] - points to the element on which the action was last performed (in arrays)
    * ApplicationGuid
    * UserDevice.IsMobile
    * UserDevice.BrowserInfo
    * UserDevice.OperatingSystem etc.
    
## 5. ACTION object
- ACTION object has methods like: `ClearDocumentRequirement()`, `ClearFieldRequirement()`,
    `SetFieldsReadOnly()` - disables editable fields, `VisibilityFlagHide(flagName)` - hide all actions with the
    flag set, `VisibilityFlagHideAll()`, `VisibilityFlagShow(flagName)` - show all actions with the flag set`,
    `VisibilityFlagShowAll()`
   
## 6. Example Functionalities:
- Setting field values based on conditions.
- Controlling field visibility, editability and requirements.
- Assigning user data to client data under specific conditions.
- Displaying or hiding screens.

## 7. Notes:
- To assign a value to a specific field, use the '=' symbol, e.g., PF.FieldInt1 = 10.
- Reference to the `.Value` property, may cause an error if the field is null.
    To avoid this, first check that the value is via `.HasValue`.
- Write a simple and easy-to-understand code. Avoid using advanced or overly complex syntax. Stick to basic structures
    and clear logic so that the code is accessible even to individuals with limited programming skills.
- Do not use comments in the code.

Always remember: respond with code only, unless instructed otherwise.

## Query:
 If the discount code has a value of "UAM5", reduce the contribution by 5 percent and display the contribution field.
""",
        "type": "rule"
    },
    {
        "input": "Jeśli członek rodziny jest studentem, obniż składkę ubezpieczenia o 10%.",
        "response" : """
for (int i = 0; i < PF.UR_DaneRodziny_T.Length; i++)
{
    if (PF.UR_DaneRodziny_T[i].DaneStudenta.NumerIndeksu.HasValue)
    {
        PF.UR_Skladka = PF.UR_Skladka * 0.9M;
        break;
    }
}
""",
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

# Universal Prompt for C# Code Generation for Client Data and Screens Management

When writing C# code, use the available process fields and screens to manage client data and user interface. Follow these guidelines:

## 1. Available Process Fields:

- PF.UR_DaneKlienta_S.Imie: String
- PF.UR_DaneKlienta_S.Nazwisko: String
- PF.UR_DaneKlienta_S.Email: String
- PF.UR_DaneKlienta_S.PESEL: String
- PF.UR_RodzajUbezpieczenia: String
- PF.UR_Skladka: Decimal
- PF.UR_PoziomUbezpieczenia: String
- PF.UR_DaneRodziny_T[].Imie: String
- PF.UR_DaneRodziny_T[].Nazwisko: String
- PF.UR_DaneRodziny_T[].Wiek: Int
- PF.UR_DaneRodziny_T[].DaneStudenta.NumerIndeksu: String
- PF.UR_DaneRodziny_T[].DaneStudenta.Kierunek: String


### Properties:
Fields have the following properties:
- `HasValue`, `IsEditable`, `IsRequired`, `IsVisible`, `Value`.

### Actions:
You can perform these operations on the fields:
- `SetEditable(bool)`, `SetVisible(bool)`, `SetRequired(bool)`, `SetNull()`, `GetValueOrDefault(defaultValue)`.

### Arrays:
- Arrays are complex datatypes that store fields or structures. Their name should end with `_T`.
    A specific element in the array can be accessed via `[index]` or `.Items(index)`.
- Operations on arrays: Length, MinimumSize (sets minimum size of an array eg. PF.Array.MinimumSize = 1)

## 2. Available Screens:
- `E010_Powitanie`
- `E020_DaneRodziny`
- `E030_WyborUbezpieczenia`
- `E040_Zgody`
- `E050_Podsumowanie`
- `E060_Odrzucenie`
- `E_techMessage`
- `Tech_SessionEndScreen`
- `Tech_TopScreen`
- `Tech_BottomScreen`
- `Tech_TabsScreen`


Do not use screen variables unless specifically asked to by user.

## Methods:
Screens can be accessed by `G.`.
Screens can be controlled using the following methods:
- `Hide()`, `HideAll()`, `Show()`, `ShowAll()`.

## 3. Accessing User Data:
- `USER.Current` provides access to properties like `IsAuthenticated`, `UserLogin`, `UserFullName`, `UserEmail`,
    `UserID`, etc.
    
## 4. ENV Object:
- ENV gives access to many environment variables like:
    * ActionIndices[0] - points to the element on which the action was last performed (in arrays)
    * ApplicationGuid
    * UserDevice.IsMobile
    * UserDevice.BrowserInfo
    * UserDevice.OperatingSystem etc.
    
## 5. ACTION object
- ACTION object has methods like: `ClearDocumentRequirement()`, `ClearFieldRequirement()`,
    `SetFieldsReadOnly()` - disables editable fields, `VisibilityFlagHide(flagName)` - hide all actions with the
    flag set, `VisibilityFlagHideAll()`, `VisibilityFlagShow(flagName)` - show all actions with the flag set`,
    `VisibilityFlagShowAll()`
   
## 6. Example Functionalities:
- Setting field values based on conditions.
- Controlling field visibility, editability and requirements.
- Assigning user data to client data under specific conditions.
- Displaying or hiding screens.

## 7. Notes:
- To assign a value to a specific field, use the '=' symbol, e.g., PF.FieldInt1 = 10.
- Reference to the `.Value` property, may cause an error if the field is null.
    To avoid this, first check that the value is via `.HasValue`.
- Write a simple and easy-to-understand code. Avoid using advanced or overly complex syntax. Stick to basic structures
    and clear logic so that the code is accessible even to individuals with limited programming skills.
- Do not use comments in the code.

Always remember: respond with code only, unless instructed otherwise.

## Query:
If a family member is a student, reduce the insurance premium by 10%.
""",
        "type": "rule"
    },
    {
        "input": "Wyświetl ekran z danymi rodziny i ustaw wszystkie dane rodziny na edytowalne.",
        "response": """
G.E020_DaneRodziny.Show();
for (int i = 0; i < PF.UR_DaneRodziny_T.Length; i++)
{
    PF.UR_DaneRodziny_T[i].Imie.SetEditable(true);
    PF.UR_DaneRodziny_T[i].Nazwisko.SetEditable(true);
    PF.UR_DaneRodziny_T[i].Wiek.SetEditable(true);
    PF.UR_DaneRodziny_T[i].Adres.Miejscowosc.SetEditable(true);
    PF.UR_DaneRodziny_T[i].Adres.Ulica.SetEditable(true);
    PF.UR_DaneRodziny_T[i].Adres.KodPocztowy.SetEditable(true);
    PF.UR_DaneRodziny_T[i].DaneStudenta.NumerIndeksu.SetEditable(true);
    PF.UR_DaneRodziny_T[i].DaneStudenta.Kierunek.SetEditable(true);
    PF.UR_DaneRodziny_T[i].DaneStudenta.Uczelnia.Nazwa.SetEditable(true);
    PF.UR_DaneRodziny_T[i].DaneStudenta.Uczelnia.Typ.SetEditable(true);
}
""",
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

# Universal Prompt for C# Code Generation for Client Data and Screens Management

When writing C# code, use the available process fields and screens to manage client data and user interface. Follow these guidelines:

## 1. Available Process Fields:

- PF.UR_DaneKlienta_S.Imie: String
- PF.UR_DaneKlienta_S.Nazwisko: String
- PF.UR_DaneKlienta_S.Email: String
- PF.UR_DaneKlienta_S.PESEL: String
- PF.UR_RodzajUbezpieczenia: String
- PF.UR_PoziomUbezpieczenia: String
- PF.UR_DaneRodziny_T[].Imie: String
- PF.UR_DaneRodziny_T[].Nazwisko: String
- PF.UR_DaneRodziny_T[].Wiek: Int
- PF.UR_DaneRodziny_T[].Adres.Miejscowosc: String
- PF.UR_DaneRodziny_T[].Adres.Ulica: String
- PF.UR_DaneRodziny_T[].Adres.KodPocztowy: String
- PF.UR_DaneRodziny_T[].DaneStudenta.NumerIndeksu: String
- PF.UR_DaneRodziny_T[].DaneStudenta.Kierunek: String
- PF.UR_DaneRodziny_T[].DaneStudenta.Uczelnia.Nazwa: String
- PF.UR_DaneRodziny_T[].DaneStudenta.Uczelnia.Typ: String
- PF.UR_DaneKlienta_S.Imie: String
- PF.UR_DaneKlienta_S.Nazwisko: String
- PF.UR_DaneKlienta_S.Email: String
- PF.UR_DaneKlienta_S.PESEL: String



### Properties:
Fields have the following properties:
- `HasValue`, `IsEditable`, `IsRequired`, `IsVisible`, `Value`.

### Actions:
You can perform these operations on the fields:
- `SetEditable(bool)`, `SetVisible(bool)`, `SetRequired(bool)`, `SetNull()`, `GetValueOrDefault(defaultValue)`.

### Arrays:
- Arrays are complex datatypes that store fields or structures. Their name should end with `_T`.
    A specific element in the array can be accessed via `[index]` or `.Items(index)`.
- Operations on arrays: Length, MinimumSize (sets minimum size of an array eg. PF.Array.MinimumSize = 1)

## 2. Available Screens:
- `E010_Powitanie`
- `E020_DaneRodziny`
- `E030_WyborUbezpieczenia`
- `E040_Zgody`
- `E050_Podsumowanie`
- `E060_Odrzucenie`
- `E_techMessage`
- `Tech_SessionEndScreen`
- `Tech_TopScreen`
- `Tech_BottomScreen`
- `Tech_TabsScreen`


Do not use screen variables unless specifically asked to by user.

## Methods:
Screens can be accessed by `G.`.
Screens can be controlled using the following methods:
- `Hide()`, `HideAll()`, `Show()`, `ShowAll()`.

## 3. Accessing User Data:
- `USER.Current` provides access to properties like `IsAuthenticated`, `UserLogin`, `UserFullName`, `UserEmail`,
    `UserID`, etc.
    
## 4. ENV Object:
- ENV gives access to many environment variables like:
    * ActionIndices[0] - points to the element on which the action was last performed (in arrays)
    * ApplicationGuid
    * UserDevice.IsMobile
    * UserDevice.BrowserInfo
    * UserDevice.OperatingSystem etc.
    
## 5. ACTION object
- ACTION object has methods like: `ClearDocumentRequirement()`, `ClearFieldRequirement()`,
    `SetFieldsReadOnly()` - disables editable fields, `VisibilityFlagHide(flagName)` - hide all actions with the
    flag set, `VisibilityFlagHideAll()`, `VisibilityFlagShow(flagName)` - show all actions with the flag set`,
    `VisibilityFlagShowAll()`
   
## 6. Example Functionalities:
- Setting field values based on conditions.
- Controlling field visibility, editability and requirements.
- Assigning user data to client data under specific conditions.
- Displaying or hiding screens.

## 7. Notes:
- To assign a value to a specific field, use the '=' symbol, e.g., PF.FieldInt1 = 10.
- Reference to the `.Value` property, may cause an error if the field is null.
    To avoid this, first check that the value is via `.HasValue`.
- Write a simple and easy-to-understand code. Avoid using advanced or overly complex syntax. Stick to basic structures
    and clear logic so that the code is accessible even to individuals with limited programming skills.
- Do not use comments in the code.

Always remember: respond with code only, unless instructed otherwise.


## Query:
Display a screen with family data and set all family data to editable.
""",
        "type": "rule",
    }
]

In [55]:
columns = [
    "input",
    "response",
    "context",
    "type",
    "NoSyntaxErrors",  
    "NoSyntaxErrorsDetails",  
    "VariableUsage",  
    "VariableUsageDetails",  
    "ClarityConciseness",  
    "ClarityConcisenessDetails",  
    "UserRequirements",  
    "UserRequirementsDetails",
]
total_score_df = pd.DataFrame(columns=columns)
for tc in test_cases:
    res = evaluate_rule(input=tc["input"], answer=tc["response"], context_prompt=tc["context"], type=tc.get('type'))
    res[0] = [tc["input"], tc["response"], tc["context"], tc["type"]] + res[0]
    result_df = pd.DataFrame(res, columns=columns)
    total_score_df = pd.concat([total_score_df, result_df], ignore_index=True)


In [56]:
total_score_df

,input,response,context,type,NoSyntaxErrors,NoSyntaxErrorsDetails,VariableUsage,VariableUsageDetails,ClarityConciseness,ClarityConcisenessDetails,UserRequirements,UserRequirementsDetails
0,"Ustaw odrzucenie przez wiek na true, jeżeli je...",\nif (PF.UR_DaneRodziny_T.Any(x => x.Wiek.HasV...,\nYou're an AI assistant on the Ferryt Low-Cod...,rule,True,,True,,True,,True,
1,Sprawdź czy w rodzinie istnieje osoba starsza ...,\nif (PF.UR_DaneRodziny_T.Any(x => x.Wiek.HasV...,\nYou're an AI assistant on the Ferryt Low-Cod...,rule,True,,True,,True,,True,
2,Ustaw minimalną długość tablicy z danymi rodzi...,\nPF.UR_DaneRodziny_T.MinimumSize = 1;\n,\nYou're an AI assistant on the Ferryt Low-Cod...,rule,True,,True,,True,,True,
3,"Jeśli kod zniżkowy ma wartość ""UAM5"" to obniż ...",\nif (PF.UR_KodZnizkowy.HasValue && PF.UR_KodZ...,\nYou're an AI assistant on the Ferryt Low-Cod...,rule,True,,False,The code does not check if PF.UR_Skladka has a...,True,,False,The code does not check if PF.UR_Skladka has a...
4,"Jeśli członek rodziny jest studentem, obniż sk...",\nfor (int i = 0; i < PF.UR_DaneRodziny_T.Leng...,\nYou're an AI assistant on the Ferryt Low-Cod...,rule,True,,True,,True,,True,
5,Wyświetl ekran z danymi rodziny i ustaw wszyst...,\nG.E020_DaneRodziny.Show();\nfor (int i = 0; ...,\nYou're an AI assistant on the Ferryt Low-Cod...,rule,True,,True,,True,,True,


# Results for rule generation (Codestral-2405)

In [57]:
true_counts = total_score_df[['NoSyntaxErrors', 'VariableUsage', 'ClarityConciseness', 'UserRequirements']].sum()
result_df = pd.DataFrame({
    'Percentage': (true_counts / len(total_score_df)) * 100
})
result_df.head()

,Percentage
NoSyntaxErrors,100.0
VariableUsage,83.333333
ClarityConciseness,100.0
UserRequirements,83.333333


# GPT-4

In [58]:
test_cases = [
    {
        "input": "Ustaw odrzucenie przez wiek na true, jeżeli jeden członek rodziny jest niepełnoletni.",
        "response": """
for(int i = 0; i < PF.UR_DaneRodziny_T.Length; i++)
{
    if(PF.UR_DaneRodziny_T[i].Wiek.Value < 18)
    {
        PF.UR_OdrzuceniePrzezWiek = true;
        break;
    }
}
        """,
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

# Universal Prompt for C# Code Generation for Client Data and Screens Management

When writing C# code, use the available process fields and screens to manage client data and user interface. Follow these guidelines:

## 1. Available Process Fields:

- PF.UR_OdrzuceniePrzezWiek: Bool
- PF.UR_DaneKlienta_S.Imie: String
- PF.UR_DaneKlienta_S.Nazwisko: String
- PF.UR_DaneKlienta_S.Email: String
- PF.UR_DaneKlienta_S.PESEL: String
- PF.UR_DaneRodziny_T[].Imie: String
- PF.UR_DaneRodziny_T[].Nazwisko: String
- PF.UR_DaneRodziny_T[].Wiek: Int



### Properties:
Fields have the following properties:
- `HasValue`, `IsEditable`, `IsRequired`, `IsVisible`, `Value`.

### Actions:
You can perform these operations on the fields:
- `SetEditable(bool)`, `SetVisible(bool)`, `SetRequired(bool)`, `SetNull()`, `GetValueOrDefault(defaultValue)`.

### Arrays:
- Arrays are complex datatypes that store fields or structures. Their name should end with `_T`.
    A specific element in the array can be accessed via `[index]` or `.Items(index)`.
- Operations on arrays: Length, MinimumSize (sets minimum size of an array eg. PF.Array.MinimumSize = 1)

## 2. Available Screens:
- `E010_Powitanie`
- `E020_DaneRodziny`
- `E030_WyborUbezpieczenia`
- `E040_Zgody`
- `E050_Podsumowanie`
- `E060_Odrzucenie`
- `E_techMessage`
- `Tech_SessionEndScreen`
- `Tech_TopScreen`
- `Tech_BottomScreen`
- `Tech_TabsScreen`


Do not use screen variables unless specifically asked to by user.

## Methods:
Screens can be accessed by `G.`.
Screens can be controlled using the following methods:
- `Hide()`, `HideAll()`, `Show()`, `ShowAll()`.

## 3. Accessing User Data:
- `USER.Current` provides access to properties like `IsAuthenticated`, `UserLogin`, `UserFullName`, `UserEmail`,
    `UserID`, etc.
    
## 4. ENV Object:
- ENV gives access to many environment variables like:
    * ActionIndices[0] - points to the element on which the action was last performed (in arrays)
    * ApplicationGuid
    * UserDevice.IsMobile
    * UserDevice.BrowserInfo
    * UserDevice.OperatingSystem etc.
    
## 5. ACTION object
- ACTION object has methods like: `ClearDocumentRequirement()`, `ClearFieldRequirement()`,
    `SetFieldsReadOnly()` - disables editable fields, `VisibilityFlagHide(flagName)` - hide all actions with the
    flag set, `VisibilityFlagHideAll()`, `VisibilityFlagShow(flagName)` - show all actions with the flag set`,
    `VisibilityFlagShowAll()`
   
## 6. Example Functionalities:
- Setting field values based on conditions.
- Controlling field visibility, editability and requirements.
- Assigning user data to client data under specific conditions.
- Displaying or hiding screens.

## 7. Notes:
- To assign a value to a specific field, use the '=' symbol, e.g., PF.FieldInt1 = 10.
- Reference to the `.Value` property, may cause an error if the field is null.
    To avoid this, first check that the value is via `.HasValue`.
- Write a simple and easy-to-understand code. Avoid using advanced or overly complex syntax. Stick to basic structures
    and clear logic so that the code is accessible even to individuals with limited programming skills.
- Do not use comments in the code.

Always remember: respond with code only, unless instructed otherwise.

## Query:
Set the age rejection to true if one family member is a minor.
        """,
        "type": "rule",
    },
    {
        "input": "Sprawdź czy w rodzinie istnieje osoba starsza niż 80 lat, jeśli tak zmień składkę podstawową na 100 * ilość osób w rodzinie.",
        "response": """
for(int i = 0; i < PF.UR_DaneRodziny_T.Length; i++)
{
    if(PF.UR_DaneRodziny_T[i].Wiek.Value > 80)
    {
        PF.UR_BasicContribution = 100 * PF.UR_DaneRodziny_T.Length;
        break;
    }
}
""",
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

# Universal Prompt for C# Code Generation for Client Data and Screens Management

When writing C# code, use the available process fields and screens to manage client data and user interface. Follow these guidelines:

## 1. Available Process Fields:

- PF.UR_OdrzuceniePrzezWiek: Bool
- PF.UR_Skladka: Decimal
- PF.UR_DaneRodziny_T[].Imie: String
- PF.UR_DaneRodziny_T[].Nazwisko: String
- PF.UR_DaneRodziny_T[].Wiek: Int
- PF.UR_DaneKlienta_S.Imie: String
- PF.UR_DaneKlienta_S.Nazwisko: String
- PF.UR_DaneKlienta_S.Email: String
- PF.UR_DaneKlienta_S.PESEL: String



### Properties:
Fields have the following properties:
- `HasValue`, `IsEditable`, `IsRequired`, `IsVisible`, `Value`.

### Actions:
You can perform these operations on the fields:
- `SetEditable(bool)`, `SetVisible(bool)`, `SetRequired(bool)`, `SetNull()`, `GetValueOrDefault(defaultValue)`.

### Arrays:
- Arrays are complex datatypes that store fields or structures. Their name should end with `_T`.
    A specific element in the array can be accessed via `[index]` or `.Items(index)`.
- Operations on arrays: Length, MinimumSize (sets minimum size of an array eg. PF.Array.MinimumSize = 1)

## 2. Available Screens:
- `E010_Powitanie`
- `E020_DaneRodziny`
- `E030_WyborUbezpieczenia`
- `E040_Zgody`
- `E050_Podsumowanie`
- `E060_Odrzucenie`
- `E_techMessage`
- `Tech_SessionEndScreen`
- `Tech_TopScreen`
- `Tech_BottomScreen`
- `Tech_TabsScreen`


Do not use screen variables unless specifically asked to by user.

## Methods:
Screens can be accessed by `G.`.
Screens can be controlled using the following methods:
- `Hide()`, `HideAll()`, `Show()`, `ShowAll()`.

## 3. Accessing User Data:
- `USER.Current` provides access to properties like `IsAuthenticated`, `UserLogin`, `UserFullName`, `UserEmail`,
    `UserID`, etc.
    
## 4. ENV Object:
- ENV gives access to many environment variables like:
    * ActionIndices[0] - points to the element on which the action was last performed (in arrays)
    * ApplicationGuid
    * UserDevice.IsMobile
    * UserDevice.BrowserInfo
    * UserDevice.OperatingSystem etc.
    
## 5. ACTION object
- ACTION object has methods like: `ClearDocumentRequirement()`, `ClearFieldRequirement()`,
    `SetFieldsReadOnly()` - disables editable fields, `VisibilityFlagHide(flagName)` - hide all actions with the
    flag set, `VisibilityFlagHideAll()`, `VisibilityFlagShow(flagName)` - show all actions with the flag set`,
    `VisibilityFlagShowAll()`
   
## 6. Example Functionalities:
- Setting field values based on conditions.
- Controlling field visibility, editability and requirements.
- Assigning user data to client data under specific conditions.
- Displaying or hiding screens.

## 7. Notes:
- To assign a value to a specific field, use the '=' symbol, e.g., PF.FieldInt1 = 10.
- Reference to the `.Value` property, may cause an error if the field is null.
    To avoid this, first check that the value is via `.HasValue`.
- Write a simple and easy-to-understand code. Avoid using advanced or overly complex syntax. Stick to basic structures
    and clear logic so that the code is accessible even to individuals with limited programming skills.
- Do not use comments in the code.

Always remember: respond with code only, unless instructed otherwise.

## Query:
Check if there is a person older than 80 years in the family; if so, change the basic contribution to 100 times the number of people in the family.
""",
        "type": "rule",
    },
    {
        "input": "Ustaw minimalną długość tablicy z danymi rodziny na 1.",
        "response": """
PF.UR_DaneRodziny_T.MinimumSize = 1;
""",
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

# Universal Prompt for C# Code Generation for Client Data and Screens Management

When writing C# code, use the available process fields and screens to manage client data and user interface. Follow these guidelines:

## 1. Available Process Fields:

- PF.UR_DaneRodziny_T[].Imie: String
- PF.UR_DaneRodziny_T[].Nazwisko: String
- PF.UR_DaneRodziny_T[].Wiek: Int



### Properties:
Fields have the following properties:
- `HasValue`, `IsEditable`, `IsRequired`, `IsVisible`, `Value`.

### Actions:
You can perform these operations on the fields:
- `SetEditable(bool)`, `SetVisible(bool)`, `SetRequired(bool)`, `SetNull()`, `GetValueOrDefault(defaultValue)`.

### Arrays:
- Arrays are complex datatypes that store fields or structures. Their name should end with `_T`.
    A specific element in the array can be accessed via `[index]` or `.Items(index)`.
- Operations on arrays: Length, MinimumSize (sets minimum size of an array eg. PF.Array.MinimumSize = 1)

## 2. Available Screens:
- `E010_Powitanie`
- `E020_DaneRodziny`
- `E030_WyborUbezpieczenia`
- `E040_Zgody`
- `E050_Podsumowanie`
- `E060_Odrzucenie`
- `E_techMessage`
- `Tech_SessionEndScreen`
- `Tech_TopScreen`
- `Tech_BottomScreen`
- `Tech_TabsScreen`


Do not use screen variables unless specifically asked to by user.

## Methods:
Screens can be accessed by `G.`.
Screens can be controlled using the following methods:
- `Hide()`, `HideAll()`, `Show()`, `ShowAll()`.

## 3. Accessing User Data:
- `USER.Current` provides access to properties like `IsAuthenticated`, `UserLogin`, `UserFullName`, `UserEmail`,
    `UserID`, etc.
    
## 4. ENV Object:
- ENV gives access to many environment variables like:
    * ActionIndices[0] - points to the element on which the action was last performed (in arrays)
    * ApplicationGuid
    * UserDevice.IsMobile
    * UserDevice.BrowserInfo
    * UserDevice.OperatingSystem etc.
    
## 5. ACTION object
- ACTION object has methods like: `ClearDocumentRequirement()`, `ClearFieldRequirement()`,
    `SetFieldsReadOnly()` - disables editable fields, `VisibilityFlagHide(flagName)` - hide all actions with the
    flag set, `VisibilityFlagHideAll()`, `VisibilityFlagShow(flagName)` - show all actions with the flag set`,
    `VisibilityFlagShowAll()`
   
## 6. Example Functionalities:
- Setting field values based on conditions.
- Controlling field visibility, editability and requirements.
- Assigning user data to client data under specific conditions.
- Displaying or hiding screens.

## 7. Notes:
- To assign a value to a specific field, use the '=' symbol, e.g., PF.FieldInt1 = 10.
- Reference to the `.Value` property, may cause an error if the field is null.
    To avoid this, first check that the value is via `.HasValue`.
- Write a simple and easy-to-understand code. Avoid using advanced or overly complex syntax. Stick to basic structures
    and clear logic so that the code is accessible even to individuals with limited programming skills.
- Do not use comments in the code.

Always remember: respond with code only, unless instructed otherwise.

## Query:
 Set the minimum length of the family data array to 1.
""",
        "type": "rule",
    },
    {
        "input": "Jeśli kod zniżkowy ma wartość \"UAM5\" to obniż składkę o 5 procent i wyświetl pole ze składką.",
        "response": """
if(PF.UR_KodZnizkowy.Value == "UAM5")
{
    PF.UR_Skladka.Value = PF.UR_Skladka.Value * 0.95m;
    PF.UR_Skladka.SetVisible(true);
}
""",
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

# Universal Prompt for C# Code Generation for Client Data and Screens Management

When writing C# code, use the available process fields and screens to manage client data and user interface. Follow these guidelines:

## 1. Available Process Fields:

- PF.UR_KodZnizkowy: String
- PF.UR_Skladka: Decimal



### Properties:
Fields have the following properties:
- `HasValue`, `IsEditable`, `IsRequired`, `IsVisible`, `Value`.

### Actions:
You can perform these operations on the fields:
- `SetEditable(bool)`, `SetVisible(bool)`, `SetRequired(bool)`, `SetNull()`, `GetValueOrDefault(defaultValue)`.

### Arrays:
- Arrays are complex datatypes that store fields or structures. Their name should end with `_T`.
    A specific element in the array can be accessed via `[index]` or `.Items(index)`.
- Operations on arrays: Length, MinimumSize (sets minimum size of an array eg. PF.Array.MinimumSize = 1)

## 2. Available Screens:
- `E010_Powitanie`
- `E020_DaneRodziny`
- `E030_WyborUbezpieczenia`
- `E040_Zgody`
- `E050_Podsumowanie`
- `E060_Odrzucenie`
- `E_techMessage`
- `Tech_SessionEndScreen`
- `Tech_TopScreen`
- `Tech_BottomScreen`
- `Tech_TabsScreen`


Do not use screen variables unless specifically asked to by user.

## Methods:
Screens can be accessed by `G.`.
Screens can be controlled using the following methods:
- `Hide()`, `HideAll()`, `Show()`, `ShowAll()`.

## 3. Accessing User Data:
- `USER.Current` provides access to properties like `IsAuthenticated`, `UserLogin`, `UserFullName`, `UserEmail`,
    `UserID`, etc.
    
## 4. ENV Object:
- ENV gives access to many environment variables like:
    * ActionIndices[0] - points to the element on which the action was last performed (in arrays)
    * ApplicationGuid
    * UserDevice.IsMobile
    * UserDevice.BrowserInfo
    * UserDevice.OperatingSystem etc.
    
## 5. ACTION object
- ACTION object has methods like: `ClearDocumentRequirement()`, `ClearFieldRequirement()`,
    `SetFieldsReadOnly()` - disables editable fields, `VisibilityFlagHide(flagName)` - hide all actions with the
    flag set, `VisibilityFlagHideAll()`, `VisibilityFlagShow(flagName)` - show all actions with the flag set`,
    `VisibilityFlagShowAll()`
   
## 6. Example Functionalities:
- Setting field values based on conditions.
- Controlling field visibility, editability and requirements.
- Assigning user data to client data under specific conditions.
- Displaying or hiding screens.

## 7. Notes:
- To assign a value to a specific field, use the '=' symbol, e.g., PF.FieldInt1 = 10.
- Reference to the `.Value` property, may cause an error if the field is null.
    To avoid this, first check that the value is via `.HasValue`.
- Write a simple and easy-to-understand code. Avoid using advanced or overly complex syntax. Stick to basic structures
    and clear logic so that the code is accessible even to individuals with limited programming skills.
- Do not use comments in the code.

Always remember: respond with code only, unless instructed otherwise.

## Query:
 If the discount code has a value of "UAM5", reduce the contribution by 5 percent and display the contribution field.
""",
        "type": "rule"
    },
    {
        "input": "Jeśli członek rodziny jest studentem, obniż składkę ubezpieczenia o 10%.",
        "response": """
for(int i = 0; i < PF.UR_DaneRodziny_T.Length; i++)
{
    if(PF.UR_DaneRodziny_T[i].DaneStudenta.HasValue)
    {
        PF.UR_Skladka = PF.UR_Skladka.Value * 0.90m;
        break;
    }
}
""",
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

# Universal Prompt for C# Code Generation for Client Data and Screens Management

When writing C# code, use the available process fields and screens to manage client data and user interface. Follow these guidelines:

## 1. Available Process Fields:

- PF.UR_DaneKlienta_S.Imie: String
- PF.UR_DaneKlienta_S.Nazwisko: String
- PF.UR_DaneKlienta_S.Email: String
- PF.UR_DaneKlienta_S.PESEL: String
- PF.UR_RodzajUbezpieczenia: String
- PF.UR_Skladka: Decimal
- PF.UR_PoziomUbezpieczenia: String
- PF.UR_DaneRodziny_T[].Imie: String
- PF.UR_DaneRodziny_T[].Nazwisko: String
- PF.UR_DaneRodziny_T[].Wiek: Int
- PF.UR_DaneRodziny_T[].DaneStudenta.NumerIndeksu: String
- PF.UR_DaneRodziny_T[].DaneStudenta.Kierunek: String


### Properties:
Fields have the following properties:
- `HasValue`, `IsEditable`, `IsRequired`, `IsVisible`, `Value`.

### Actions:
You can perform these operations on the fields:
- `SetEditable(bool)`, `SetVisible(bool)`, `SetRequired(bool)`, `SetNull()`, `GetValueOrDefault(defaultValue)`.

### Arrays:
- Arrays are complex datatypes that store fields or structures. Their name should end with `_T`.
    A specific element in the array can be accessed via `[index]` or `.Items(index)`.
- Operations on arrays: Length, MinimumSize (sets minimum size of an array eg. PF.Array.MinimumSize = 1)

## 2. Available Screens:
- `E010_Powitanie`
- `E020_DaneRodziny`
- `E030_WyborUbezpieczenia`
- `E040_Zgody`
- `E050_Podsumowanie`
- `E060_Odrzucenie`
- `E_techMessage`
- `Tech_SessionEndScreen`
- `Tech_TopScreen`
- `Tech_BottomScreen`
- `Tech_TabsScreen`


Do not use screen variables unless specifically asked to by user.

## Methods:
Screens can be accessed by `G.`.
Screens can be controlled using the following methods:
- `Hide()`, `HideAll()`, `Show()`, `ShowAll()`.

## 3. Accessing User Data:
- `USER.Current` provides access to properties like `IsAuthenticated`, `UserLogin`, `UserFullName`, `UserEmail`,
    `UserID`, etc.
    
## 4. ENV Object:
- ENV gives access to many environment variables like:
    * ActionIndices[0] - points to the element on which the action was last performed (in arrays)
    * ApplicationGuid
    * UserDevice.IsMobile
    * UserDevice.BrowserInfo
    * UserDevice.OperatingSystem etc.
    
## 5. ACTION object
- ACTION object has methods like: `ClearDocumentRequirement()`, `ClearFieldRequirement()`,
    `SetFieldsReadOnly()` - disables editable fields, `VisibilityFlagHide(flagName)` - hide all actions with the
    flag set, `VisibilityFlagHideAll()`, `VisibilityFlagShow(flagName)` - show all actions with the flag set`,
    `VisibilityFlagShowAll()`
   
## 6. Example Functionalities:
- Setting field values based on conditions.
- Controlling field visibility, editability and requirements.
- Assigning user data to client data under specific conditions.
- Displaying or hiding screens.

## 7. Notes:
- To assign a value to a specific field, use the '=' symbol, e.g., PF.FieldInt1 = 10.
- Reference to the `.Value` property, may cause an error if the field is null.
    To avoid this, first check that the value is via `.HasValue`.
- Write a simple and easy-to-understand code. Avoid using advanced or overly complex syntax. Stick to basic structures
    and clear logic so that the code is accessible even to individuals with limited programming skills.
- Do not use comments in the code.

Always remember: respond with code only, unless instructed otherwise.

## Query:
If a family member is a student, reduce the insurance premium by 10%.
""",
        "type": "rule"
    },
    {
        "input": "Wyświetl ekran z danymi rodziny i ustaw wszystkie dane rodziny na edytowalne.",
        "response": """
G.E020_DaneRodziny.Show();

for(int i = 0; i < PF.UR_DaneRodziny_T.Length; i++)
{
    PF.UR_DaneRodziny_T[i].Imie.SetEditable(true);
    PF.UR_DaneRodziny_T[i].Nazwisko.SetEditable(true);
    PF.UR_DaneRodziny_T[i].Wiek.SetEditable(true);
    PF.UR_DaneRodziny_T[i].Adres.Miejscowosc.SetEditable(true);
    PF.UR_DaneRodziny_T[i].Adres.Ulica.SetEditable(true);
    PF.UR_DaneRodziny_T[i].Adres.KodPocztowy.SetEditable(true);
    if(PF.UR_DaneRodziny_T[i].DaneStudenta.HasValue)
    {
        PF.UR_DaneRodziny_T[i].DaneStudenta.NumerIndeksu.SetEditable(true);
        PF.UR_DaneRodziny_T[i].DaneStudenta.Kierunek.SetEditable(true);
        PF.UR_DaneRodziny_T[i].DaneStudenta.Uczelnia.Nazwa.SetEditable(true);
        PF.UR_DaneRodziny_T[i].DaneStudenta.Uczelnia.Typ.SetEditable(true);
    }
}
""",
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

# Universal Prompt for C# Code Generation for Client Data and Screens Management

When writing C# code, use the available process fields and screens to manage client data and user interface. Follow these guidelines:

## 1. Available Process Fields:

- PF.UR_DaneKlienta_S.Imie: String
- PF.UR_DaneKlienta_S.Nazwisko: String
- PF.UR_DaneKlienta_S.Email: String
- PF.UR_DaneKlienta_S.PESEL: String
- PF.UR_RodzajUbezpieczenia: String
- PF.UR_PoziomUbezpieczenia: String
- PF.UR_DaneRodziny_T[].Imie: String
- PF.UR_DaneRodziny_T[].Nazwisko: String
- PF.UR_DaneRodziny_T[].Wiek: Int
- PF.UR_DaneRodziny_T[].Adres.Miejscowosc: String
- PF.UR_DaneRodziny_T[].Adres.Ulica: String
- PF.UR_DaneRodziny_T[].Adres.KodPocztowy: String
- PF.UR_DaneRodziny_T[].DaneStudenta.NumerIndeksu: String
- PF.UR_DaneRodziny_T[].DaneStudenta.Kierunek: String
- PF.UR_DaneRodziny_T[].DaneStudenta.Uczelnia.Nazwa: String
- PF.UR_DaneRodziny_T[].DaneStudenta.Uczelnia.Typ: String
- PF.UR_DaneKlienta_S.Imie: String
- PF.UR_DaneKlienta_S.Nazwisko: String
- PF.UR_DaneKlienta_S.Email: String
- PF.UR_DaneKlienta_S.PESEL: String



### Properties:
Fields have the following properties:
- `HasValue`, `IsEditable`, `IsRequired`, `IsVisible`, `Value`.

### Actions:
You can perform these operations on the fields:
- `SetEditable(bool)`, `SetVisible(bool)`, `SetRequired(bool)`, `SetNull()`, `GetValueOrDefault(defaultValue)`.

### Arrays:
- Arrays are complex datatypes that store fields or structures. Their name should end with `_T`.
    A specific element in the array can be accessed via `[index]` or `.Items(index)`.
- Operations on arrays: Length, MinimumSize (sets minimum size of an array eg. PF.Array.MinimumSize = 1)

## 2. Available Screens:
- `E010_Powitanie`
- `E020_DaneRodziny`
- `E030_WyborUbezpieczenia`
- `E040_Zgody`
- `E050_Podsumowanie`
- `E060_Odrzucenie`
- `E_techMessage`
- `Tech_SessionEndScreen`
- `Tech_TopScreen`
- `Tech_BottomScreen`
- `Tech_TabsScreen`


Do not use screen variables unless specifically asked to by user.

## Methods:
Screens can be accessed by `G.`.
Screens can be controlled using the following methods:
- `Hide()`, `HideAll()`, `Show()`, `ShowAll()`.

## 3. Accessing User Data:
- `USER.Current` provides access to properties like `IsAuthenticated`, `UserLogin`, `UserFullName`, `UserEmail`,
    `UserID`, etc.
    
## 4. ENV Object:
- ENV gives access to many environment variables like:
    * ActionIndices[0] - points to the element on which the action was last performed (in arrays)
    * ApplicationGuid
    * UserDevice.IsMobile
    * UserDevice.BrowserInfo
    * UserDevice.OperatingSystem etc.
    
## 5. ACTION object
- ACTION object has methods like: `ClearDocumentRequirement()`, `ClearFieldRequirement()`,
    `SetFieldsReadOnly()` - disables editable fields, `VisibilityFlagHide(flagName)` - hide all actions with the
    flag set, `VisibilityFlagHideAll()`, `VisibilityFlagShow(flagName)` - show all actions with the flag set`,
    `VisibilityFlagShowAll()`
   
## 6. Example Functionalities:
- Setting field values based on conditions.
- Controlling field visibility, editability and requirements.
- Assigning user data to client data under specific conditions.
- Displaying or hiding screens.

## 7. Notes:
- To assign a value to a specific field, use the '=' symbol, e.g., PF.FieldInt1 = 10.
- Reference to the `.Value` property, may cause an error if the field is null.
    To avoid this, first check that the value is via `.HasValue`.
- Write a simple and easy-to-understand code. Avoid using advanced or overly complex syntax. Stick to basic structures
    and clear logic so that the code is accessible even to individuals with limited programming skills.
- Do not use comments in the code.

Always remember: respond with code only, unless instructed otherwise.


## Query:
Display a screen with family data and set all family data to editable.
""",
        "type": "rule",
    }
]
columns = [
    "input",
    "response",
    "context",
    "type",
    "NoSyntaxErrors",
    "NoSyntaxErrorsDetails",
    "VariableUsage",
    "VariableUsageDetails",
    "ClarityConciseness",
    "ClarityConcisenessDetails",
    "UserRequirements",
    "UserRequirementsDetails",
]
total_score_df = pd.DataFrame(columns=columns)
for tc in test_cases:
    res = evaluate_rule(input=tc["input"], answer=tc["response"], context_prompt=tc["context"], type=tc.get('type'))
    res[0] = [tc["input"], tc["response"], tc["context"], tc["type"]] + res[0]
    result_df = pd.DataFrame(res, columns=columns)
    total_score_df = pd.concat([total_score_df, result_df], ignore_index=True)


In [59]:
total_score_df

,input,response,context,type,NoSyntaxErrors,NoSyntaxErrorsDetails,VariableUsage,VariableUsageDetails,ClarityConciseness,ClarityConcisenessDetails,UserRequirements,UserRequirementsDetails
0,"Ustaw odrzucenie przez wiek na true, jeżeli je...",\nfor(int i = 0; i < PF.UR_DaneRodziny_T.Lengt...,\nYou're an AI assistant on the Ferryt Low-Cod...,rule,True,,False,The code does not check if the 'Wiek' field ha...,True,,True,
1,Sprawdź czy w rodzinie istnieje osoba starsza ...,\nfor(int i = 0; i < PF.UR_DaneRodziny_T.Lengt...,\nYou're an AI assistant on the Ferryt Low-Cod...,rule,False,The field 'PF.UR_BasicContribution' does not e...,True,,True,,False,The code does not meet the user requirements b...
2,Ustaw minimalną długość tablicy z danymi rodzi...,\nPF.UR_DaneRodziny_T.MinimumSize = 1;\n,\nYou're an AI assistant on the Ferryt Low-Cod...,rule,True,,True,,True,,True,
3,"Jeśli kod zniżkowy ma wartość ""UAM5"" to obniż ...","\nif(PF.UR_KodZnizkowy.Value == ""UAM5"")\n{\n ...",\nYou're an AI assistant on the Ferryt Low-Cod...,rule,False,The code does not check if PF.UR_KodZnizkowy o...,True,,True,,False,The code does not check if PF.UR_KodZnizkowy o...
4,"Jeśli członek rodziny jest studentem, obniż sk...",\nfor(int i = 0; i < PF.UR_DaneRodziny_T.Lengt...,\nYou're an AI assistant on the Ferryt Low-Cod...,rule,True,,True,,True,,True,
5,Wyświetl ekran z danymi rodziny i ustaw wszyst...,\nG.E020_DaneRodziny.Show();\n\nfor(int i = 0;...,\nYou're an AI assistant on the Ferryt Low-Cod...,rule,True,,True,,True,,True,


# Results for rule generation (GPT-4)

In [60]:
true_counts = total_score_df[['NoSyntaxErrors', 'VariableUsage', 'ClarityConciseness', 'UserRequirements']].sum()
result_df = pd.DataFrame({
    'Percentage': (true_counts / len(total_score_df)) * 100
})
result_df.head()

,Percentage
NoSyntaxErrors,66.666667
VariableUsage,83.333333
ClarityConciseness,100.0
UserRequirements,66.666667
